In [1]:
%matplotlib widget

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import flammkuchen as fl

from glob import glob
import pandas as pd
from bouter import Experiment
import colorspacious
import json
from scipy import stats

In [3]:
master_pre = Path(r"Z:\Hagar\E0040\v31\pre ablation")
pre_list = list(master_pre.glob("*_f*"))

master_post = Path(r"Z:\Hagar\E0040\v31\post ablation 2d")
post_list = list(master_post.glob("*_f*"))

num_fish = len(pre_list)

In [ ]:
###### make a figure showing the drop per fish

In [5]:
regions = ['r_habenula', 'l_habenula', 'ahb', 'ipn']
num_regions = len(regions)
thresh = 0.33
n_tuned_pre = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN
n_neurons_pre = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN
n_tuned_post = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN
n_neurons_post = np.zeros((num_regions, num_fish)) # habenula, aHB, IPN

In [7]:
bins = np.linspace(-1, 1, 100)    
fig2, ax2 = plt.subplots(num_regions, num_fish, figsize=(12, 7))

for fish in range(num_fish):
    pre_fish = pre_list[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    traces = fl.load(pre_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(pre_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(pre_fish / "registration" / "ref_brain_regions.h5")
    
    for region in range(num_regions):
        print(regions[region])
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        traces_region = traces[:, region_idx]

        reg_corr = np.zeros((2, num_traces))

        count=0
        for direction in [0, 4]:
            current_dir = np.asarray(sensory_regressors.iloc[:, direction])

            for i in range(num_traces):
                reg_corr[count, i] = np.corrcoef(current_dir, traces_region[:, i])[0,1]
            count += 1
        
        n_neurons_pre[region, fish] = num_traces
        n_tuned_pre[region, fish] = len(np.where(np.abs(reg_corr) > thresh)[0])
        ax2[region, fish].hist(np.append(reg_corr[0], reg_corr[1]), bins, alpha=0.5, label='pre', density=True)
        ax2[region, 0].set_ylabel("Count(" + regions[region] + ")")
    
    ax2[0, fish].set_title(fish_id)
    ax2[0, fish].set_xlabel("Correlation")
    
    #######################################
    post_fish = post_list[fish]
    
    traces = fl.load(post_fish / "filtered_traces.h5", "/detr")
    sensory_regressors = fl.load(post_fish / "sensory_regressors.h5", "/regressors")
    coords_regions = fl.load(post_fish / "registration" / "ref_brain_regions.h5")
        
    for region in range(num_regions):
        region_idx = coords_regions[regions[region]]
        num_traces = np.shape(region_idx)[0]
        traces_region = traces[:, region_idx]
        
        reg_corr = np.zeros((2, num_traces))
        count=0
        for direction in [0, 4]:
            current_dir = np.asarray(sensory_regressors.iloc[:, direction])

            for i in range(num_traces):
                reg_corr[count, i] = np.corrcoef(current_dir, traces_region[:, i])[0,1]
            count += 1

        n_neurons_post[region, fish] = num_traces
        n_tuned_post[region, fish] = len(np.where(np.abs(reg_corr) > thresh)[0])
        ax2[region, fish].hist(np.append(reg_corr[0], reg_corr[1]), bins, alpha=0.5, label='post', density=True)
        ax2[0, num_fish-1].legend(loc='upper right')
        
        ax2[region, fish].spines['right'].set_visible(False)
        ax2[region, fish].spines['top'].set_visible(False)
   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

230424_f0
r_habenula
l_habenula
ahb
ipn


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


230424_f1
r_habenula
l_habenula
ahb
ipn


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


...

230424_f4
r_habenula
l_habenula
ahb
ipn


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


230424_f5
r_habenula
l_habenula
ahb
ipn


No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.


230424_f6
r_habenula
l_habenula
ahb
ipn


In [8]:
fig2.subplots_adjust(left=0.05, wspace=0.3)


In [9]:
file_name = "corr histogram pre and post norm morphed masks.jpg"
fig2.savefig(master_pre / file_name, dpi=300)
file_name = "corr histogram pre and post norm morphed masks.pdf"
fig2.savefig(master_pre / file_name, dpi=300)


In [13]:
titles = ['Left habenula', 'Right habenula', 'aHB', 'IPN']

In [14]:
fig3, ax3 = plt.subplots(2, num_regions, figsize=(8, 5))
ylim=[[0.2, 0.1, 1, 0.4], [1,1,1,1]]
for j in range(2):
    for i in range(num_regions):
        curr_plot = np.zeros((2, num_fish))
        curr_plot[0] = n_tuned_pre[i] / n_neurons_pre[i] / 2
        curr_plot[1] = n_tuned_post[i] / n_neurons_post[i] / 2
        ax3[j, i].plot(curr_plot, c='skyblue')
        x = np.mean(curr_plot, axis=1)
        err = np.std(curr_plot) / num_fish
        ax3[j, i].errorbar([0, 1], x, yerr=err, c='royalblue')


        ax3[0, i].set_title(titles[i])
        ax3[j, i].spines['right'].set_visible(False)
        ax3[j, i].spines['top'].set_visible(False)

        labels = [item.get_text() for item in ax3[j, i].get_xticklabels()]
        labels[1] = 'Pre'
        labels[2] = 'Post'

        ax3[j, i].set_xticklabels(labels)
        pval = round(stats.ttest_rel(curr_plot[0], curr_plot[1])[1], 4)
        ax3[j, i].set_ylim(0, ylim[j][i])
        ax3[j, i].text(0.3, ylim[j][i]*0.9, "p=" + str(pval), fontsize=9)


    ax3[j, 0].set_ylabel('Tuned')
fig3.subplots_adjust(wspace=0.5, hspace=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:22: UserWarning: FixedFormatter should only be used together with FixedLocator
C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel

In [12]:
file_name = "precent change in tuned neurons new" + str(thresh) + " morphed masks.jpg"
fig3.savefig(master_pre / file_name, dpi=300)
file_name = "precent change in tuned neurons new" + str(thresh) + " morphed masks.pdf"
fig3.savefig(master_pre / file_name, dpi=300)

In [90]:
for fish in range(1):
    pre_fish = pre_list[fish]
    exp = Experiment(pre_fish)
    fish_id = exp.fish_id
    print(fish_id)
    
    coords = fl.load(pre_fish / "data_from_suite2p_cells.h5", "/coords")
    
    trans_mat = fl.load(pre_fish / 'registration' / 'initial_transform_mapped.h5')
    print(trans_mat)


230424_f0
[[ 8.73623388e-01 -4.64943531e-02 -2.30999630e-02 -2.36872900e+01]
 [ 1.15950053e-02  9.93257537e-01  1.37209535e-01 -1.27431027e+02]
 [ 2.40065541e-03 -1.17106349e-01  1.08772746e+00 -7.19318644e+01]]


In [91]:
import tifffile as tiff

In [92]:
ref = tiff.imread(r"Z:\Hagar\mpin_zfish_1um_v1.1\H2BGCaMP.tiff")

In [101]:
np.shape(ref)

(974, 359, 597)

In [108]:
ref2 = np.swapaxes(ref, 0,1)

In [111]:
ref3 = np.rot90(ref2)

In [ ]:
ref4 = ref3[:,:,:]

In [112]:
tiff.imsave(path / "test.tiff", ref3)

In [98]:
path = Path(r"Z:\Hagar\mpin_zfish_1um_v1.1")
d = {'ref_mapped': ref}
fl.save(path / "ref_mapped.h5", d)